# Aprendizagem Automática Avançada
## TP7 - BAYES exercises
João Romão - 55760,
Pedro França - 55848

## 0. Approach to solving the exercise

In order to solve the exercise, we decided to model the Bayesian Network shown in the [assignement diagram](https://www.ida.liu.se/ext/caisor/TDDC65/dectree-exercise/page-100930.html) using the pomegranate library. This library allows us to model the network using a straightforward approach, where we first define the probabilites of each node, for each state (T or F, which we will represent as 1 and 0, respectively), based on the state of their parent nodes, as presented in the diagram. We then established the parent-child relation (edges) for the different pairs of nodes and constructed the final model.

## 1. Modeling the network

To create the Bayesian Network model we followed the instruction from two different tutorial using pomegranate ([tutorial1](https://github.com/jmschrei/pomegranate/blob/master/tutorials/B_Model_Tutorial_4_Bayesian_Networks.ipynb) and [tutorial2](https://github.com/jmschrei/pomegranate/blob/master/tutorials/B_Model_Tutorial_4b_Bayesian_Network_Structure_Learning.ipynb)), which are referenced in the [pomegranate documentation for Bayesian Networks](https://pomegranate.readthedocs.io/en/latest/BayesianNetwork.html).

In [1]:
from pomegranate import *

In [2]:
# Defining the probabilities for each node of the Bayesian Network, according to the diagram

A = DiscreteDistribution({1: 0.30, 0: 0.70})
    
B = ConditionalProbabilityTable([ [0, 1, 0.4],
                                  [0, 0, 0.6],
                                  [1, 1, 0.8],
                                  [1, 0, 0.2]], [A])
    
C = ConditionalProbabilityTable([[0,0,1, 0.1],
                                 [0,0,0, 0.9],
                                 [0,1,1, 0.7],
                                 [0,1,0, 0.3],
                                 [1,0,1, 0.5],
                                 [1,0,0, 0.5],
                                 [1,1,1, 0.99],
                                 [1,1,0, 0.01]],[A, B])

D = ConditionalProbabilityTable([ [0, 1, 0.55],
                                  [0, 0, 0.45],
                                  [1, 1, 0.2],
                                  [1, 0, 0.8]], [B])

In [3]:
# Instantiating each node
s1 = Node(A, name="A")
s2 = Node(B, name="B")
s3 = Node(C, name="C")
s4 = Node(D, name="D")

In [4]:
# Create the Bayesian network object with a useful name
model = BayesianNetwork("Causal Net")

# Add the four states to the network 
model.add_states(s1, s2, s3 ,s4)

In [5]:
# Defining the endges between parent and child nodes
model.add_edge(s1, s2)
model.add_edge(s1, s3)
model.add_edge(s2, s3)
model.add_edge(s2, s4)
model.bake()

## 2. Calculating the queries

After creating the model, we can calculate the conditional probabilities by using the method `.predict_proba()`. For example, if we wish to calculate P(A=T|B=T), we can use `model.predict_proba({'B':1})` and then inspect the resulting array to retrieve the probability corresponding to A=T (or {'A':1}). We will solve the first exercise to illustrate the strategy:

### 1) P(A=T|C=T,D=T) 

In [6]:
prob = model.predict_proba({'C':1, 'D':1})
prob

array([{
    "class" : "Distribution",
    "dtype" : "int",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "1" : 0.5054138717420109,
            "0" : 0.49458612825798914
        }
    ],
    "frozen" : false
},
       {
    "class" : "Distribution",
    "dtype" : "int",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "0" : 0.3516850950686168,
            "1" : 0.6483149049313832
        }
    ],
    "frozen" : false
},
       1, 1], dtype=object)

The `.predict_proba()` returns an array with four elements, corresponding to the four nodes we defined ('A', 'B', 'C', and 'D'). We can see that the last two elements of the array are 1 and 1, corresponding to the states we defined for 'C' and 'D'. The first two positions correspond to 'A' and 'B', respectively, and each of those contains a dictionary inside of which we can inspect the probabilities for '1' and '0', as the value for the 'parameters' key. Therefore, if we wish to know P(A=T|C=T,D=T), and we already established that (C=T,D=T), we need to check the first element (corresponding to 'A') of the array for the boolean state '1', in order to know that the probability we are calculating is __0.5054138717420109__.

Although effective in practice, we are aware that this is not the most elegant way to display our results, so we defined the 'result' function to retrieve that value for us. We only need to pass 3 parameters: the result of the `.predict_proba()`, the node we wish to predict the probability given `.predict_proba()`, and the state of that node (0 or 1). The function will access the corresponding position of the letter in the array, transform it to a dictionary, and return the value for the boolean state key.

In [7]:
def result(predict, letter, TF):
    #changing the letter to an integer n, according to its alphabetical position
    letter_to_int = {'A':0, 'B':1, 'C':2, 'D':3}
    n = letter_to_int[letter]
    
    # retrieving the n'th position of the prediction array, corresponding to the letter position
    n_position = predict[n]
    
    # converting the element in the n'th position to an str and processing it so it becomes a
    # string representation of a dictionary
    dict_prob_str = str(n_position)
    dict_prob_str = dict_prob_str.replace('\n','')
    dict_prob_str = dict_prob_str.replace(' ', '')
    dict_prob_str = dict_prob_str.replace('false', '"false"')
    
    # converting the string representation of a dictionary to a dictionary
    dict_prob = eval(dict_prob_str)
    
    # accessing the dictionary inside the 'parameters' key and retrieving the value (probability)
    # corresponding to the TF key that was passed (0 or 1)
    prob_value = dict_prob['parameters'][0][str(TF)]
    
    return prob_value

In [8]:
prob = model.predict_proba({'C':1, 'D':1})
p1 = result(prob, 'A', 1)
print("Probability result:" ,p1)

Probability result: 0.5054138717420109


After confirming that our function works as it should, we simply repeat the process for the remaining exercises.

### 2) P(A=T|D=F)

In [9]:
prob = model.predict_proba({'D':0})
p2 = result(prob, 'A', 1)
print("Probability result:" ,p2)

Probability result: 0.34651898734177244


### 3) P(B=T|C=T)

In [10]:
prob = model.predict_proba({'C':1})
p3 = result(prob, 'B', 1)
print("Probability result:" ,p3)

Probability result: 0.8100843263425553


### 4) P(B=T|A=T,C=T) 

In [11]:
prob = model.predict_proba({'A':1 , 'C':1})
p4 = result(prob, 'B', 1)
print("Probability result:" ,p4)

Probability result: 0.8878923766816139


### 5) P(C=T|A=F,B=F,D=F) 

In [12]:
prob = model.predict_proba({'A':0 ,'B':0, 'D':0})
p5 = result(prob, 'C', 1)
print("Probability result:" ,p5)

Probability result: 0.10000000000000016


## 3. Summary of results

In [13]:
print("1) " + str(round(p1,4)))
print("2) " + str(round(p2,4)))
print("3) " + str(round(p3,4)))
print("4) " + str(round(p4,4)))
print("5) " + str(round(p5,4)))

1) 0.5054
2) 0.3465
3) 0.8101
4) 0.8879
5) 0.1


## 4. Results using Hugin Lite 8.9

We also used the Hugin Lite 8.9 software to solve the exercise and got the results bellow:

    H1) 0.5068
    H2) 0.3465
    H3) 0.8576
    H4) 0.8879
    H5) 0.1000

We included the output images from the Hugin software in the pdf document attached to this submission. We can see that the results for exercises 2, 4, and 5 were the same using pomegranate or Hugin, wile the results for 1 and 3 were, respectively, 0.014 and 0.475 higher when using Hugin. We are not certain as to why these differences occur, namely the magnitude of difference we see in 3. We will not discuss divergences in the results using the different softwares, as we simply wanted to disclose them.